<a href="https://colab.research.google.com/github/marianayarce1224/lab_biosenales/blob/main/PROYECTO3/Proyecto3_G3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left"><img alt="udeA logo" height="150px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true"></p>

<h1><b>Proyecto 3: Filtros</b></h1>
<h4><b>Bioseñales y sistemas<br>Universidad de Antioquia, Facultad de Ingeniería - Bioingeniería<br>2024-2</b></h4><br>

<h4><b>Grupo:</b> 3</h4>
<h4><b>Integrantes:</b> Mariana Álvarez Yarce - Ana María Seguro Acevedo</h4>

##<b>Contexto del problema</b>

<h4>Las enfermedades cardiovasculares son una de las principales causas de muerte global, con 17,9 millones de muertes en 2019 según la OMS. Su riesgo se incrementa por factores como el tabaquismo, hipertensión, sedentarismo, obesidad, entre otras condiciones.</h4>

<h4>La tecnología, incluyendo sensores en dispositivos de Electrocardiografía (ECG), puede mejorar tratamientos y seguimientos. La medición de parámetros como frecuencia cardíaca y presión arterial es esencial, aunque su variabilidad requiere datos de grandes muestras para construir modelos diagnósticos precisos. El avance respecto a bioseñales implica organizar la cantidad de datos necesaria sobre diversas enfermedades cardiovasculares, mediante un procesado y etiquetado adecuados, con el fin de construir modelos para diagnóstico o predicción. El enfoque de grandes volúmenes de datos permite validar el procesamiento y análisis en diferentes poblaciones y tecnologías, aumentando la pertinencia de las tecnologías desarrolladas.</h4>

<h4>El ECG, que registra la actividad eléctrica del corazón, es fundamental en la detección de problemas cardiovasculares. Se utiliza un sistema de 12 derivaciones, combinando electrodos en extremidades y tórax para obtener una visión integral de la función cardíaca, con amplitudes de señal de 10 µV a 4 mV y frecuencias relevantes de 0,05 a 100 Hz <b>[1]</b>.</h4>

##<b>Origen de los datos</b>

<h4>La base de datos para el desarrollo del proyecto se basa en una investigación de ECG, establecida por la Universidad de Chapman y el Hospital del Pueblo de Shaoxing. La investigación tiene por objetivo estudiar las señales biomédicas cardiovasculares a través de extensas simulaciones por computadora. Aborda condiciones como la fibrilación auricular, que impactan significativamente en la salud pública y los costos médicos. La base de datos incluye 10,646 ECG de 12 derivaciones muestreados a 500 Hz, con 11 ritmos comunes y 67 condiciones <b>[2]</b>.</h4>

##<b>Entregable</b>

<h4><b><u>Nota:</u></b> Los análisis para el presente proyecto deben hacerse en la derivación II de los registros seleccionados.</h4>

###<h4><b>1. </b>Del artículo <b>[3]</b>, consultar y explicar los dos métodos de reducción de ruido usados en el artículo: Robust LOESS y Non Local Means. Mostrar cómo se podrían implementar en Python.</h4>

<b>Robust LOESS (Locally Estimated Scatterplot Smoothing):</b> Es un método de reducción de ruido aplicable a señales no paramétricas, que funciona con el ajuste de una función de regresión para una señal a partir de una ventana deslizante. La ventana toma diferentes valores locales y realiza un ajuste por mínimos cuadrados ponderados, de acuerdo a un parámetro de suavización $α$, que se puede establecer en el siguiente rango:

<h4>$(λ + 1)/n \lt α \lt 1$</h4>

<b>Donde</b>
> $λ$ es el grado del polinomio local<br>
$n$ es el número de datos de la señal

Valores más cercanos a la unidad, resultan en curvas más suaves, mientras que valores pequeños del parámetro pueden implicar un sobreajuste de la curva a los datos. Se recomienda definir $\alpha$ en un rango aproximado de 0,25 a 0,5.

Esta técnica es relativamente resistente a valores atípicos o ruido en los datos, teniendo en cuenta el tipo de regresión que se utilice (por lo general se usa la regresión polinómica de grado 2 en adelante).

> <b>Pasos para Robust LOESS:</b><br>
* Selección de vecindad local: Para cada punto, se define un vecindario de puntos cercanos (con base en la distancia o una ventana de tamaño fijo).
* Ponderación: Se calculan pesos de los puntos en función de su distancia al punto central de la vecindad. Los puntos más cercanos tienen más peso.
* Ajuste de regresión local: Se ajusta una regresión polinómica (típicamente de primer o segundo orden) a los puntos dentro del vecindario.
* Robustez: Los puntos atípicos son identificados y se les da un peso bajo para que no afecten el modelo final.

Implementación en Python:
La librería statsmodels en Python ofrece una implementación de LOESS. Para usar la versión robusta, se puede especificar el parámetro adecuado. Aquí hay un ejemplo:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Datos de ejemplo
np.random.seed(42)
x = np.linspace(0, 10, 100)
y = np.sin(x) + np.random.normal(0, 0.2, 100)  # Sinusoidal con ruido

# Aplicar Robust LOESS
lowess = sm.nonparametric.lowess(y, x, frac=0.2, it=3)  # 'frac' es la fracción de puntos cercanos, 'it' es el número de iteraciones de robustez
y_smoothed = lowess[:, 1]

# Visualizar
plt.figure(figsize=(10, 6))
plt.scatter(x, y, label="Datos originales", alpha=0.6)
plt.plot(x, y_smoothed, label="LOESS suave", color='red', linewidth=2)
plt.legend()
plt.title('Aplicación de Robust LOESS')
plt.show()

Non-Local Means (NLM)
El algoritmo Non-Local Means es un método de reducción de ruido basado en la idea de que el valor de un píxel en una imagen (o en general, de un punto de datos) puede ser estimado de manera más precisa utilizando la información de otros píxeles/vecinos que son similares a él, no solo los puntos cercanos en el espacio, sino cualquier punto que sea similar a él en términos de sus características.

En NLM, la "no localidad" se refiere al hecho de que el valor de un punto se suaviza usando información de puntos lejanos, siempre y cuando estos sean similares.

Pasos para Non-Local Means:
Definir un vecindario: Para cada punto, se comparan sus vecinos en un área amplia (no necesariamente cercana).
Calcular similitudes: Se calculan las similitudes entre el punto central y los demás puntos usando una métrica como la distancia Euclidiana en un espacio de características (por ejemplo, en imágenes, se puede usar la distancia entre los valores de intensidad de píxeles).
Ponderación: Los puntos más similares tienen mayor peso en la estimación del valor central, mientras que los puntos menos similares tienen un peso bajo.
Suavizado: El valor estimado de un punto se calcula como la media ponderada de los valores de los puntos vecinos.
Implementación en Python:
El algoritmo Non-Local Means está ampliamente utilizado en procesamiento de imágenes, y se puede encontrar en la librería cv2 de OpenCV, que ofrece una función para realizarlo. Aquí hay un ejemplo de cómo usarlo en Python:

cv2.fastNlMeansDenoising: Esta función de OpenCV implementa el algoritmo Non-Local Means. Los parámetros son:
La imagen de entrada (con ruido).
None: La imagen de salida (se generará una nueva imagen).
30: Parámetro de suavizado, controla la cantidad de ruido que se elimina.
7: El tamaño del vecindario (en píxeles).
21: El tamaño del filtro (también en términos de vecindad, controla cuántos puntos se consideran).
Resumen de las Implementaciones en Python:
Robust LOESS: Usa una ventana local y aplica una regresión ponderada. En Python, puedes usar statsmodels para implementarlo.
Non-Local Means: Suaviza la imagen o los datos utilizando la similitud entre puntos lejanos. En Python, la librería OpenCV (cv2.fastNlMeansDenoising) es ideal para implementarlo.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Crear una imagen de ejemplo con ruido
image = np.ones((256, 256), dtype=np.uint8) * 100  # Imagen uniforme de valor 100
image[50:150, 50:150] = 200  # Agregar un área más brillante
noisy_image = image + np.random.normal(0, 30, image.shape).astype(np.uint8)

# Aplicar Non-Local Means para reducir el ruido
denoised_image = cv2.fastNlMeansDenoising(noisy_image, None, 30, 7, 21)

# Visualizar los resultados
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(noisy_image, cmap='gray')
plt.title('Imagen con Ruido')

plt.subplot(1, 2, 2)
plt.imshow(denoised_image, cmap='gray')
plt.title('Imagen después de Non-Local Means')

plt.show()

###<h4><b>2. </b>De la base de datos extraer los registros que correspondan a bradicardia sinusal (<i>SB Sinus Bradycardia</i>) y fibrilación auricular (<i>AFIB Atrial Fibrillation</i>).</h4>



Para fines prácticos de este proyecto, se filtraron los datos de acuerdo a los valores de la columna <i>Rhythm</i> correspondientes a <i>SB</i> o <i>AFIBB</i>. Este proceso, junto con la eliminación de las columnas diferentes de la derivación II en los archivos .csv, se realizó a través del código en el siguiente <a href="https://colab.research.google.com/drive/1sMFf2cTo8-dqhe6M75dvQLkE-cEPaHih?usp=sharing" target="_blank">enlace</a>.

Posteriormente, se hizo un lista de todos los registros, tanto para SB como para AFIB, para acceder a los datos a través de rutas cuando se requiera. Esto es más manejable y no consume tantos recursos computacionales cada vez que se ejecuta el código.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, fft, stats
from pathlib import Path
import seaborn as sns

ruta_SB= '/content/drive/MyDrive/ECGData/ECGData_SB/'
lista_archivos_SB = glob.glob(ruta_SB + '*.csv')
ruta_AFIB = '/content/drive/MyDrive/ECGData/ECGData_AFIB/'
lista_archivos_AFIB = glob.glob(ruta_AFIB + '*.csv')
print(f'Archivos SB: {lista_archivos_SB}')
print(f'Archivos AFIB: {lista_archivos_AFIB}')
print(f'\nCantidad archivos = {len(lista_archivos_SB) + len(lista_archivos_AFIB)}')

###<h4><b>3. </b>Consultar qué otros tipos de señales wavelet se pueden usar para el análisis de señales ECG y adaptar el código del filtro wavelet que se entrega en el curso de acuerdo a la consulta.</h4>

###<h4><b>4. </b>Escoger 10 señales al azar y aplicar un flujo de procesamiento que conste de:</h4>

<blockquote><h4><b>Flujo 1:</b></h4><blockquote>
<h4><b>1.</b> Filtro pasa-altas usando filtro IIR a 0.5 Hz. Justificar la elección de parámetros y si se usa FIR o IIR<br>
<b>2.</b> Filtro wavelet modificado del punto 3<br>
<b>3.</b> Filtrado pasabajas 50 Hz. Justificar la elección de parámetros y si se usa FIR o IIR</h4>
</blockquote></blockquote>

<blockquote><h4><b>Flujo 2:</b></h4><blockquote>
<h4><b>1.</b> Detrend<br>
<b>2.</b> Filtro wavelet modificado del punto 3<br>
<b>3.</b> Filtrado pasabajas 50 Hz. Justificar la elección de parámetros y si se usa FIR o IIR</h4>
</blockquote></blockquote>

<blockquote><h4><b>Flujo 3:</b></h4><blockquote>
<h4><b>1.</b> Filtro pasa-altas usando filtro IIR a 0.5 Hz. Justificar la elección de parámetros y si se usa FIR o IIR<br>
<b>2.</b> Filtrado pasabajas 50 Hz. Justificar la elección de parámetros y si se usa FIR o IIR<br>
<b>3.</b> Describir los resultados obtenidos y decidir si el resto del procesamiento se hace con el flujo 1, flujo 2 o flujo 3</h4>
</blockquote></blockquote>

<h4>Con el flujo seleccionado procesar las señales de la base de datos <i>ECGData.zip</i> (derivación II). En este punto deberíamos estar en el mismo punto donde partimos en el proyecto 2, con señales filtradas.</h4>

<h4>A las señales filtradas con el flujo seleccionado aplicar el procesamiento de normalización usado en el proyecto 2.</h4>

In [ ]:
def compresion(signal):
    signal = np.array(signal)
    x_min = np.min(signal)
    x_max = np.max(signal)
    s_tk = (signal - x_min) / (x_max - x_min)
    return s_tk

###<h4><b>5. </b>Para cada señal extraer la frecuencia que contiene la máxima potencia usando Welch.</h4>

###<h4><b>6. </b>Crear una rutina que aplique sobre todos los archivos de la base de datos las rutina 3 al 5 y almacene los resultados en un dataframe donde se pueda registro, tipo de patología y el frecuencia de máxima potencia (fMP).</h4>

###<h4><b>7. </b>Comparar los resultados de fMP del proyecto 3 con los del proyecto 2 usando estadística descriptiva: gráficos y pruebas de hipótesis. Discuta si hay más diferencias entre los tipos de señales con el flujo de procesamiento propuesto respecto al que se trabajó en el proyecto 2.</h4>

###<h4><b>8. </b>Hacer un informe con todos los puntos anteriores.</h4>

<b><u>Discusión</u></b><br>

<b><u>Conclusiones</u></b>

##<b>Referencias</b>


<b>[1]:</b> Biosenales y Sistemas. (3 de Octubre, 2024). Proyecto3 en <i>proyecto</i> [Repositorio de GitHub]. biosenalesysistemas. https://github.com/biosenalesysistemas/proyecto

<b>[2]:</b> Zheng, Jianwei (2019). ChapmanECG. figshare. Collection. https://doi.org/10.6084/m9.figshare.c.4560497.v1

<b>[3]:</b> Zheng, J., Chu, H., Struppa, D. et al. Optimal Multi-Stage Arrhythmia Classification Approach. <i>Sci Rep</i> 10, 2898 (2020). https://doi.org/10.1038/s41598-020-59821-7

<b>[4]:</b> Wikipedia. (5 de Diciembre, 2023). <i>Regresión local</i>. Wikipedia, La enciclopedia libre. Recuperado de https://es.wikipedia.org/wiki/Regresi%C3%B3n_local

<b>[5]:</b>